In [51]:
import requests
from PIL import Image
import imagehash
import pandas as pd
import ipyplot
import glob
import distance
#https://practicaldatascience.co.uk/data-science/how-to-use-image-hashing-to-identify-visually-similar-or-duplicate-images#:~:text=Image%20hashing%20has%20two%20main,each%20image%20to%20check%20it.

In [52]:
pd.set_option('display.expand_frame_repr', False)

In [53]:
images = []

path = '/Users/danielbernardo/Desktop/Image_test_folder/*.*'

for file in glob.glob(path):
    print(file)

/Users/danielbernardo/Desktop/Image_test_folder/A2_0003.png
/Users/danielbernardo/Desktop/Image_test_folder/X1_0025.png
/Users/danielbernardo/Desktop/Image_test_folder/X1_0019.png
/Users/danielbernardo/Desktop/Image_test_folder/N35_Z2_0001.png
/Users/danielbernardo/Desktop/Image_test_folder/G17_0007.png
/Users/danielbernardo/Desktop/Image_test_folder/V11_0002.png
/Users/danielbernardo/Desktop/Image_test_folder/G17_0013.png
/Users/danielbernardo/Desktop/Image_test_folder/V10_0002.png
/Users/danielbernardo/Desktop/Image_test_folder/Z7_0017.png
/Users/danielbernardo/Desktop/Image_test_folder/Z7_0003.png
/Users/danielbernardo/Desktop/Image_test_folder/Z2_0009.png
/Users/danielbernardo/Desktop/Image_test_folder/G39_A1_0001.png
/Users/danielbernardo/Desktop/Image_test_folder/Z2_0008.png
/Users/danielbernardo/Desktop/Image_test_folder/Z7_0002.png
/Users/danielbernardo/Desktop/Image_test_folder/Z7_0016.png
/Users/danielbernardo/Desktop/Image_test_folder/G17_0012.png
/Users/danielbernardo/Deskt

In [54]:
df = pd.DataFrame(columns=['image','ahash','phash','dhash','whash','colorhash'])


for file in glob.glob(path):
    file = Image.open(file)

    data = {
        'image': file,
        'ahash': imagehash.average_hash(file),
        'phash': imagehash.phash(file),
        'dhash': imagehash.dhash(file),
        'whash': imagehash.whash(file),
        'colorhash': imagehash.colorhash(file),   
    }
    
    df = df.append(data, ignore_index=True)

/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_71333/2877828194.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_71333/2877828194.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_71333/2877828194.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_71333/2877828194.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.

In [55]:
df.head(10)

,image,ahash,phash,dhash,whash,colorhash
0,<PIL.PngImagePlugin.PngImageFile image mode=RG...,a333372f9f2001bf,afe56069f922c68c,4766676d70696f60,a333372f9f0001bf,17000000000
1,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f9f3c380e0f1c71f,eae889ca5b38cab1,21060f1301030f78,f9f3c380c0e1c31f,17000000000
2,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f9e3cf0100f0f38f,fac2c84a1fca9ab2,41871c3f0001871c,f8e3cf0100f0f38f,1e000000000
3,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f080fdf3ef0100fc,fae95a35e045889b,011d71061c330839,f080fdf3ef0100fc,1e000000000
4,<PIL.PngImagePlugin.PngImageFile image mode=RG...,c0f0e701f1f1f30f,ec7ad4483a3791e2,01230f0f0303073c,c0f0e301f1f1f30f,17000000000
5,<PIL.PngImagePlugin.PngImageFile image mode=RG...,37270f0f8f83909f,bf83c46c994cb199,664d5a5b182f2320,33270f0f8f83909f,17000000000
6,<PIL.PngImagePlugin.PngImageFile image mode=RG...,e787df8000f9f38f,bab3b76216c4dc30,0d1fb0244163077c,e787df0000f1f30f,17000000000
7,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f1f0fcfcf9f3c71f,ca4a25ed8ab685b9,2301000913860c72,e0f0f8f8f0e1830f,0e000000000
8,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f8e7c3c0f8f3c71f,ea7295cd4a70b199,014f8f0343871c73,f8c3c1c0f0f1871f,0e000000000
9,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fcf081818181071f,bbc890eb8ab68ab2,410005254b471e79,fdf0c18181838f3f,33000000000


In [56]:
def find_similar_images(df, ahash_column, input_image):
    """Compare an unseen image to previously seen images and return
    a list of images ranked by their similarity according to the 
    Hamming distance of their average hash or ahash.
    
    :param df: Pandas dataframe containing image and ahash columns
    :param ahash_column: Name of ahash column
    :param image_url: URL of the unseen image to hash and compare
   
    :return
        Pandas dataframe containing the most similar images
    """
    
    input = Image.open(input_image)
    ahash = str(imagehash.average_hash(input))
        
    df['hamming_distance'] = df.apply(\
    lambda x: distance.hamming(str(x[ahash_column]), ahash), axis=1)

    df = df[['image','ahash','hamming_distance']]\
    .sort_values(by='hamming_distance', ascending=True)
    
    return df

In [57]:
df = find_similar_images(df, 'ahash', '/Users/danielbernardo/Desktop/Image_test_folder/B1_0005.png')

In [58]:
pd.set_option("display.max_columns", 100)

In [59]:
df.head(10)

,image,ahash,hamming_distance
126,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fdf9d3830040c7cf,0
195,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fdf9e3830040c0e3,4
1,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f9f3c380e0f1c71f,8
356,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fff38180000017bf,8
292,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fcf3e38300c0f1e7,8
268,<PIL.PngImagePlugin.PngImageFile image mode=RG...,f7b38303030080cf,9
391,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fbf7cf80000099ff,9
13,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fcf9f0f8f9f3c71f,10
80,<PIL.PngImagePlugin.PngImageFile image mode=RG...,fdfbf7ef800005ff,10
32,<PIL.PngImagePlugin.PngImageFile image mode=RG...,e0f9f381f0f8f38f,10
